<a href="https://colab.research.google.com/github/ArshiaSarkar/IBM-project/blob/main/Stylist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q opencv-python matplotlib pillow
!pip install -q git+https://github.com/facebookresearch/segment-anything.git
!pip install -q git+https://github.com/huggingface/diffusers.git
!pip install -q transformers accelerate
!pip install -q gradio
!pip install -q scikit-image

import gradio as gr
import torch
import numpy as np
import cv2
from PIL import Image
from skimage.morphology import closing, square
from segment_anything import sam_model_registry, SamPredictor
from segment_anything.utils.amg import remove_small_regions
from diffusers import StableDiffusionInpaintPipeline

import os
sam_checkpoint = "sam_vit_b_01ec64.pth"
if not os.path.exists(sam_checkpoint):
    !wget -q https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth


device = "cuda" if torch.cuda.is_available() else "cpu"


model_type = "vit_b"
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint).to(device)
predictor = SamPredictor(sam)

pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16 if device=="cuda" else torch.float32,
    use_safetensors=False,
).to(device)

def resize_image(image, max_size=512):
    w, h = image.size
    if max(w, h) > max_size:
        scale = max_size / max(w, h)
        new_w, new_h = int(w * scale), int(h * scale)
        image = image.resize((new_w, new_h), Image.LANCZOS)
    return image

def smooth_mask(mask):
    closed = closing(mask, square(7))
    return closed

def get_best_mask(masks):
    areas = [m.sum() for m in masks]
    max_idx = np.argmax(areas)
    return masks[max_idx]

def validate_occasion(occasion):
    allowed = ["wedding", "office party", "casual outing", "formal event", "cocktail", "birthday party", "business meeting", "date night", "vacation", "festival"]
    occ = occasion.lower().strip()
    if occ not in allowed:
        return None, f"Invalid occasion! Allowed: {', '.join(allowed)}"
    return occ, None

def ai_stylist(image, occasion):
    occasion_clean, err = validate_occasion(occasion)
    if err:
        return None, err

    image = resize_image(image, max_size=512)

    image_bgr = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    predictor.set_image(image_bgr)

    h, w = image.size[1], image.size[0]
    input_box = np.array([[10, 10, w - 10, h - 10]])

    masks, _, _ = predictor.predict(box=input_box, multimask_output=True)

    best_mask = get_best_mask(masks)

    clean_mask, _ = remove_small_regions(best_mask, area_thresh=1024, mode="islands")
    smooth = smooth_mask(clean_mask.astype(bool))

    mask_array = smooth.astype(bool)

    np_image = np.array(image)
    if np.mean(np_image[mask_array]) > np.mean(np_image[~mask_array]):
        mask_array = ~mask_array


    mask_image = Image.fromarray((mask_array * 255).astype(np.uint8)).convert("RGB")

    # Compose more detailed prompt for inpainting
    prompt = (
        f"Fashionable, realistic, high-quality full-body outfit for a {occasion_clean}. "
        "Include stylish shirt, pants or skirt, and shoes. "
        "Modern, trendy, well-fitted clothing with vibrant colors and clean background."
    )
    negative_prompt = (
        "blurry, low-resolution, distorted anatomy, cropped, extra limbs, "
        "bad outfit, messy, poorly dressed, out of frame"
    )

    # Run Stable Diffusion inpainting with error handling
    try:
        result = pipe(
            prompt=prompt,
            negative_prompt=negative_prompt,
            image=image,
            mask_image=mask_image,
            guidance_scale=8.0,
            num_inference_steps=60,
        ).images[0]
    except Exception as e:
        return None, f"Inpainting error: {e}"

    return result, None

# Gradio UI
iface = gr.Interface(
    fn=ai_stylist,
    inputs=[
        gr.Image(type="pil", label="Upload Your Full-Body Photo"),
        gr.Textbox(lines=1, placeholder="Enter occasion (e.g. wedding, office party)", label="Occasion"),
    ],
    outputs=[
        gr.Image(type="pil", label="Styled Outfit"),
        gr.Textbox(label="Status", interactive=False)
    ],
    title="Smart AI Stylist: Dress Up for Your Occasion",
    description="Upload your photo, enter a valid occasion, and get a styled outfit tailored for you!",
    allow_flagging="never",
    live=False
)

iface.launch(share=True)


  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.7 MB/s eta 0:00:00


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/548 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c1a5d2ce9e3fa82a4b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
